# Metaprogramming: "Code that creates code" 

Julia has strong metaprogramming capabilities, i.e. we can use Julia to manipulate objects in Julia that represent Julia code. In this way, we can produce code in an automatic way.

Since this is rather abstract, let's consider a very concrete example: Wilkinson-type polynomials.
The [Wilkinson polynomial](https://en.wikipedia.org/wiki/Wilkinson's_polynomial) is

$$p_{20}(x) := (x-1) \cdot (x-2) \cdot \cdots \cdot (x-20) = \prod_{i=1}^{20} (x-i)$$

[Polynomials like this are interesting since their eigenvalues are very sensitive to perturbations of the coefficients of the polynomial.]

Suppose we wish to define this polynomial in Julia. The simple way would be to write it explicitly:

In [1]:
p_5(x) = (x-1)*(x-2)*(x-3)*(x-4)*(x-5)

p_5 (generic function with 1 method)

$p_{10}$ is already a pain to type by hand, $p_{20}$ more so, and $p_{100}$ is basically impossible. But this is just a case of repetition, and computers are designed for that. Of course, one possible definition uses a `for` loop:

In [2]:
function wilkinson(n, x)
    result = x - 1
    
    for i in 2:n
        result *= (x - i)
    end
    
    result
end

wilkinson (generic function with 1 method)

We can use an anonymous function to have the actual function object $p_n$:

In [3]:
wilkinson(n) = x -> wilkinson(n, x)

wilkinson (generic function with 2 methods)

However, as we have seen, currently "anonymous functions are slow". It seems like it should be possible to use the original definition of the function by "unrolling the loop" and writing a loop that writes *the code to generate the function*.

## Expressions 

In other languages, e.g. Python, code is manipulated as strings. Julia takes a different approach. Consider the string

In [4]:
s = "(x-1) * (x-2)"

"(x-1) * (x-2)"

To convert this into an object that represents Julia code, we **parse** it:

In [5]:
ex = parse(s)

:((x - 1) * (x - 2))

In [6]:
typeof(ans)

Expr

`ex` is a Julia *expression object*. It can be thought of as a representation of the "abstract syntax tree" (AST) representing the internal structure of the expression. We can see this in two ways, using `dump`:

In [7]:
dump(ex)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol *
    2: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol -
        2: Symbol x
        3: Int64 1
      typ: Any
    3: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol -
        2: Symbol x
        3: Int64 2
      typ: Any
  typ: Any


which shows everything in detail, or 

In [8]:
Meta.show_sexpr(ex)

(:call, :*, (:call, :-, :x, 1), (:call, :-, :x, 2))

which gives a compact version.

We see that `Expr`s have a hierarchical format that represents the code. Since they are simply Julia objects, e.g. `Array`s, we *can use Julia to manipulate them*.

The object `:x` is of type Symbol, and represents the unevaluated object `:x`. This is basically a special type of string.

**Exercise**: Write a function that takes an expression object and replaces all the `:x`s by `:z`s.

## Code interpolation

In our case, we do not need to mess with the internal structure of code, but rather build up code from preexisting bits. We can do this as follows:

In [24]:
ex = :(x-1)

:(x - 1)

In [25]:
ex = :(($ex) * (x-2))

:((x - 1) * (x - 2))

Here, in the same way as in string interpolation, we have inserted the current *value* of `ex` into the expression!

In [26]:
ex = :(($ex) * (x-3))

:(((x - 1) * (x - 2)) * (x - 3))

Now we can make our loop:

In [37]:
n = 10
ex = :(x-1)
for i in 2:n
    ex = :( ($ex) * (x-i) )
end

In [38]:
ex

:((((((((((x - 1) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i))

This did not work, since we did not want "the code '`i`'", but rather the value of `i`. So:

In [39]:
n = 10
ex = :(x-1)
for i in 2:n
    ex = :( ($ex) * (x-$i) )
end

In [40]:
ex

:((((((((((x - 1) * (x - 2)) * (x - 3)) * (x - 4)) * (x - 5)) * (x - 6)) * (x - 7)) * (x - 8)) * (x - 9)) * (x - 10))

Now we need to produce the name of the function:

In [34]:
name = symbol(string("W_", n))

:W_10

[In Julia v0.4, this can be written more simply as `symbol("W_", n)`.]

So the code that we would like is

In [41]:
code = :( $name(x) = $ex )

:(W_10(x) = (((((((((x - 1) * (x - 2)) * (x - 3)) * (x - 4)) * (x - 5)) * (x - 6)) * (x - 7)) * (x - 8)) * (x - 9)) * (x - 10))

Now we wish to evaluate this:

In [42]:
eval(code)

W_10 (generic function with 1 method)

This creates a function with the name `W_10` that does (almost) exactly what we would write by hand.

Let's compare the two options by evaluating the function on a grid of values:

In [43]:

f1(range) = [W_10(x) for x in range]
f2(range) = [wilkinson(10, x) for x in range]

f2 (generic function with 1 method)

In [55]:
range = -10:0.000001:10
@time f1(range);
@time f2(range);

elapsed time: 0.641817258 seconds (160000144 bytes allocated)
elapsed time: 1.210885709 seconds (160000144 bytes allocated)


We see that the generated code with the unrolled loop is twice as fast as the naive `for` loop.

Code generation like this is used frequently in Julia code when repetitive code is required.

## Macros

The things that feel like functions whose names start with `@` that we have been using, such as `@time`, `@which`, etc., are not functions in the standard sense, but rather are **macros**. These are "super-functions" whose arguments are code expressions, and which transform one code expression into another. This new code expression is then evaluated as if the new code had been typed directly.

To understand what is going on, let's define a simple macro:

In [58]:
macro simple(expr)
    @show expr
    0  # for the moment
end

In [57]:
@simple x+y

expr => :(x + y)


0

We see that the Julia code that follows the macro call is passed to the macro *already having been parsed into an `Expr` object*.

Suppose we redefine the macro as

In [59]:
macro simple(expr)
    @show expr
    expr  # returns expr
end

Then we get

In [60]:
@simple x+y

expr => :(x + y)


LoadError: x not defined
while loading In[60], in expression starting on line 1

What is happening here is that the macro returns the expression `:(x+y)`, and this is then evaluated using `eval`.
The result is that Julia tries to calculate the value of the expression `x+y`, but the variable `x` is not defined. Let's define `y` and `z`, but not `x`:

In [61]:
y = 3; z = 4

4

In [62]:
x

LoadError: x not defined
while loading In[62], in expression starting on line 1

In [63]:
@simple x+y

expr => :(x + y)


LoadError: x not defined
while loading In[63], in expression starting on line 1

Now let's define a new macro `@replace` that uses our previous `replace` function:

In [66]:
macro replace!(expr)
    replace(expr)
    @show expr
    expr
end

In [67]:
@replace! x+y

expr => :(z + y)


7

The macro has transformed the original expression into the new one, which is then evaluated.

Thus macros allow us to take valid Julia code and transform it in arbitrarily complicated ways.
This is used in many packages to allow the creation of **domain-specific languages** (DSLs) -- macros provide a means of adding new syntax to the language in a sense.